## Conversion of simulations for analysis in RevBayes

I need to write a file in Rev to run an analysis in RevBayes

The first part imports some libraries

In [1]:
import sys
import re

In [2]:
calibration_templ = """
tmrca_NODENAME := tmrca(timetree,NODENAME)
n_TMRCA_NODENAME := -(tmrca_NODENAME)
NODENAME_fossil_age <- -FOSSILAGE
NODENAME_true_age <- TRUENODEAGE
NODENAME_lambda <- 1.0 / (NODENAME_true_age - FOSSILAGE)
NODENAME_fossil_snode ~ dnExponential(lambda=NODENAME_lambda, offset=n_TMRCA_NODENAME)
NODENAME_fossil_snode.clamp(NODENAME_fossil_age)
"""